## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 20th Oct. 27th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-22-2024,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
67,PAST SALE,2024-10-25,Condo/Co-op,11650 SW 2nd St #16107,Pembroke Pines,FL,33025.0,335000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/11650...,MARMLS,A11573200,N,Y,26.004719,-80.303703
14,PAST SALE,2024-10-25,Condo/Co-op,255 S Cypress Rd #335,Pompano Beach,FL,33060.0,115000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/255-S-...,MARMLS,A11650823,N,Y,26.228275,-80.124225
28,PAST SALE,2024-10-25,Condo/Co-op,2225 SW 15th St #231,Deerfield Beach,FL,33442.0,265000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/2225...,Beaches MLS,F10461305,N,Y,26.297443,-80.134179
26,PAST SALE,2024-10-25,Condo/Co-op,4812 N State Road 7 #105,Coconut Creek,FL,33073.0,245000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coconut-Creek/4812-N...,Beaches MLS,F10452127,N,Y,26.288122,-80.201150
45,PAST SALE,2024-10-25,Condo/Co-op,900 SW 12th St #200,Fort Lauderdale,FL,33315.0,240000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/900-...,Beaches MLS,F10453246,N,Y,26.106897,-80.153679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,PAST SALE,2024-10-21,Condo/Co-op,2775 Taft St #203,Hollywood,FL,33020.0,171000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2775-Taft-...,MARMLS,A11625757,N,Y,26.026517,-80.162935
69,PAST SALE,2024-10-21,Condo/Co-op,2301 S Ocean Dr #2405,Hollywood,FL,33019.0,575000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2301-S-Oce...,MARMLS,A11591260,N,Y,25.997517,-80.116976
68,PAST SALE,2024-10-21,Condo/Co-op,110 SE 2nd St #306,Hallandale Beach,FL,33009.0,170000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/110...,MARMLS,A11596245,N,Y,25.983024,-80.147197
17,PAST SALE,2024-10-21,Condo/Co-op,2700 NE 56th Ct Unit 2B,Fort Lauderdale,FL,33308.0,592500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2700...,MARMLS,A11659850,N,Y,26.197028,-80.109948


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

98


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$303,409


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$277


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$29,734,100


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
111
Input Previous Week Condo Average Sales Price:
302175
Input Previous Week Condo Average PSF:
276
Input Previous Week Condo Sales total (ex: 198_000_000)
33_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange
1,PAST SALE,2024-10-24,Condo/Co-op,48 Hendricks Isle Unit 3A-A,Fort Lauderdale,FL,33301.0,1400000.0,3.0,3.5,The Meritage,2458.0,NaN,2006.0,NaN,570.0,1913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Beaches MLS,F10433988,N,Y,26.122099,-80.121742,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Matthew Bertanzetti,Florida Luxurious Properties,None,None,Spencer Zorn,"Compass Florida, LLC.",None,None,2,blue
2,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #3808,Hallandale Beach,FL,33009.0,1085000.0,3.0,2.5,BEACH CLUB CONDO,1571.0,NaN,2005.0,NaN,691.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11650323,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Damira Liberman,LoKation,None,None,Boris Gelfand,"Gelfand Realty, Inc.",None,None,3,blue
3,PAST SALE,2024-10-23,Condo/Co-op,1631 Riverview Rd #808,Deerfield Beach,FL,33441.0,800000.0,3.0,2.0,Hillsboro Landings Condo NO One,1824.0,NaN,1973.0,NaN,439.0,983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1631...,Beaches MLS,RX-11006751,N,Y,26.313508,-80.084597,https://www.redfin.com/FL/Deerfield-Beach/1631...,Jill Kendall,RE/MAX Select Group,Judith Wheeler,RE/MAX Select Group,Gustavo Pavone,The Cove Guspav Realty,None,None,4,blue
4,PAST SALE,2024-10-24,Condo/Co-op,600 W Las Olas Blvd #703,Fort Lauderdale,FL,33312.0,735000.0,2.0,2.0,Symphony South,1348.0,NaN,2005.0,NaN,545.0,1397.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Beaches MLS,F10448050,N,Y,26.118290,-80.149165,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Austin Bergman,"Compass Florida, LLC",None,None,None,None,None,None,5,blue
5,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #2106,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB CONDO,1086.0,NaN,2005.0,NaN,645.0,1119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11589107,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Marta Kottar,One Sotheby's International Realty,None,None,6,blue
6,PAST SALE,2024-10-21,Condo/Co-op,1201 S Ocean Dr Unit 2504N,Hollywood,FL,33019.0,692000.0,2.0,2.0,The Summit,1570.0,NaN,1982.0,NaN,441.0,1564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Beaches MLS,F10450988,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Jill Fugger,BW Oceanside,None,None,Mariene Menin,Coldwell Banker Realty,None,None,7,blue
7,PAST SALE,2024-10-24,Condo/Co-op,5200 N Ocean Blvd #1203,Lauderdale By The Sea,FL,33308.0,685000.0,2.0,2.0,Sea Ranch Lakes North,1330.0,NaN,1971.0,NaN,515.0,943.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/5200...,Beaches MLS,F10463114,N,Y,26.201000,-80.094629,https://www.redfin.com/FL/Fort-Lauderdale/5200...,Eldrin DiBello,"By The Sea Realty, Inc.",None,None,Eldrin DiBello,"By The Sea Realty, Inc.",None,None,8,blue
8,PAST SALE,2024-10-24,Condo/Co-op,4106 W Palm Aire Dr Unit 63a,Pompano Beach,FL,33069.0,685000.0,2.0,2.5,Vizcaya Towers,2586.0,NaN,1986.0,NaN,265.0,1318.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beac

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_10282024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'45 HENDRICKS CONDO at 45 Hendricks Isle #201 in Fort Lauderdale'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange,45 HENDRICKS CONDO at 45 Hendricks Isle #201 i...
1,PAST SALE,2024-10-24,Condo/Co-op,48 Hendricks Isle Unit 3A-A,Fort Lauderdale,FL,33301.0,1400000.0,3.0,3.5,The Meritage,2458.0,NaN,2006.0,NaN,570.0,1913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Beaches MLS,F10433988,N,Y,26.122099,-80.121742,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Matthew Bertanzetti,Florida Luxurious Properties,None,None,Spencer Zorn,"Compass Florida, LLC.",None,None,2,blue,The Meritage at 48 Hendricks Isle Unit 3A-A in...
2,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #3808,Hallandale Beach,FL,33009.0,1085000.0,3.0,2.5,BEACH CLUB CONDO,1571.0,NaN,2005.0,NaN,691.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11650323,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Damira Liberman,LoKation,None,None,Boris Gelfand,"Gelfand Realty, Inc.",None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3808 in H...
3,PAST SALE,2024-10-23,Condo/Co-op,1631 Riverview Rd #808,Deerfield Beach,FL,33441.0,800000.0,3.0,2.0,Hillsboro Landings Condo NO One,1824.0,NaN,1973.0,NaN,439.0,983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1631...,Beaches MLS,RX-11006751,N,Y,26.313508,-80.084597,https://www.redfin.com/FL/Deerfield-Beach/1631...,Jill Kendall,RE/MAX Select Group,Judith Wheeler,RE/MAX Select Group,Gustavo Pavone,The Cove Guspav Realty,None,None,4,blue,Hillsboro Landings Condo NO One at 1631 Riverv...
4,PAST SALE,2024-10-24,Condo/Co-op,600 W Las Olas Blvd #703,Fort Lauderdale,FL,33312.0,735000.0,2.0,2.0,Symphony South,1348.0,NaN,2005.0,NaN,545.0,1397.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Beaches MLS,F10448050,N,Y,26.118290,-80.149165,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Austin Bergman,"Compass Florida, LLC",None,None,None,None,None,None,5,blue,Symphony South at 600 W Las Olas Blvd #703 in ...
5,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #2106,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB CONDO,1086.0,NaN,2005.0,NaN,645.0,1119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11589107,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Marta Kottar,One Sotheby's International Realty,None,None,6,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #2106 in H...
6,PAST SALE,2024-10-21,Condo/Co-op,1201 S Ocean Dr Unit 2504N,Hollywood,FL,33019.0,692000.0,2.0,2.0,The Summit,1570.0,NaN,1982.0,NaN,441.0,1564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Beaches MLS,F10450988,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Jill Fugger,BW Oceanside,None,None,Mariene Menin,Coldwell Banker Realty,None,None,7,blue,The Summit at 1201 S Ocean Dr Unit 2504N in Ho...
7,PAST SALE,2024-10-24,Condo/Co-op,5200 N Ocean Blvd #1203,Lauderdale By The Sea,FL,33308.0,685000.0,2.0,2.0,Sea Ranch Lakes North,1330.0,NaN,1971.0,NaN,515.0,943.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/5200...,Beaches MLS,F10463114,N,Y,26.201000,-80.094629,https:

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 45 HENDRICKS CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $592,500M to $1,800,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hallandale Beach, Pompano Beach, Deerfield Beach, Fort Lauderdale, Lauderdale By The Sea, Hollywood

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 98 condo sales totaling $29,734,100 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 111 condo sales totaling $33,500,000.

Last week’s units sold for an average of $303,409, higher than the $302,175 average price from the previous we

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange,45 HENDRICKS CONDO at 45 Hendricks Isle #201 i...
1,PAST SALE,2024-10-24,Condo/Co-op,48 Hendricks Isle Unit 3A-A,Fort Lauderdale,FL,33301.0,1400000.0,3.0,3.5,The Meritage,2458.0,NaN,2006.0,NaN,570.0,1913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Beaches MLS,F10433988,N,Y,26.122099,-80.121742,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Matthew Bertanzetti,Florida Luxurious Properties,None,None,Spencer Zorn,"Compass Florida, LLC.",None,None,2,blue,The Meritage at 48 Hendricks Isle Unit 3A-A in...
2,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #3808,Hallandale Beach,FL,33009.0,1085000.0,3.0,2.5,BEACH CLUB CONDO,1571.0,NaN,2005.0,NaN,691.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11650323,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Damira Liberman,LoKation,None,None,Boris Gelfand,"Gelfand Realty, Inc.",None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3808 in H...
3,PAST SALE,2024-10-23,Condo/Co-op,1631 Riverview Rd #808,Deerfield Beach,FL,33441.0,800000.0,3.0,2.0,Hillsboro Landings Condo NO One,1824.0,NaN,1973.0,NaN,439.0,983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1631...,Beaches MLS,RX-11006751,N,Y,26.313508,-80.084597,https://www.redfin.com/FL/Deerfield-Beach/1631...,Jill Kendall,RE/MAX Select Group,Judith Wheeler,RE/MAX Select Group,Gustavo Pavone,The Cove Guspav Realty,None,None,4,blue,Hillsboro Landings Condo NO One at 1631 Riverv...
4,PAST SALE,2024-10-24,Condo/Co-op,600 W Las Olas Blvd #703,Fort Lauderdale,FL,33312.0,735000.0,2.0,2.0,Symphony South,1348.0,NaN,2005.0,NaN,545.0,1397.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Beaches MLS,F10448050,N,Y,26.118290,-80.149165,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Austin Bergman,"Compass Florida, LLC",None,None,None,None,None,None,5,blue,Symphony South at 600 W Las Olas Blvd #703 in ...
5,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #2106,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB CONDO,1086.0,NaN,2005.0,NaN,645.0,1119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11589107,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Marta Kottar,One Sotheby's International Realty,None,None,6,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #2106 in H...
6,PAST SALE,2024-10-21,Condo/Co-op,1201 S Ocean Dr Unit 2504N,Hollywood,FL,33019.0,692000.0,2.0,2.0,The Summit,1570.0,NaN,1982.0,NaN,441.0,1564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Beaches MLS,F10450988,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Jill Fugger,BW Oceanside,None,None,Mariene Menin,Coldwell Banker Realty,None,None,7,blue,The Summit at 1201 S Ocean Dr Unit 2504N in Ho...
7,PAST SALE,2024-10-24,Condo/Co-op,5200 N Ocean Blvd #1203,Lauderdale By The Sea,FL,33308.0,685000.0,2.0,2.0,Sea Ranch Lakes North,1330.0,NaN,1971.0,NaN,515.0,943.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/5200...,Beaches MLS,F10463114,N,Y,26.201000,-80.094629,https:

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: 45 HENDRICKS CONDO closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $592,500M to $1,800,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hallandale Beach, Pompano Beach, Deerfield Beach, Fort Lauderdale, Lauderdale By The Sea, Hollywood

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 98 condo sales totaling $29,734,100 million from Oct. 20th to Oct. 27th. The previous week, brokers closed 111 condo sales totaling $33,500,000.

Last week’s units sold for an average of $303,409, higher than the $302,175 average price from the previous we

In [55]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Fort-Lauderdale/45-Hendricks-Isle-33301/unit-201/home/42061833


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/48-Hendricks-Isle-33301/unit-3A-A/home/190498745


In [63]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/2700-NE-56th-Ct-33308/unit-2B/home/41652062


In [64]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Pompano-Beach/4106-W-Palm-Aire-Dr-33069/unit-63A/home/41643863


In [65]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Hallandale-Beach/1850-S-Ocean-Dr-33009/unit-3808/home/42041075


In [66]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/4106-W-Palm-Aire-Dr-33069/unit-63A/home/41643863


In [67]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-22,Condo/Co-op,45 Hendricks Isle #201,Fort Lauderdale,FL,33301.0,1800000.0,3.0,3.5,45 HENDRICKS CONDO,2979.0,NaN,2007.0,NaN,604.0,2379.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,MARMLS,A11611277,N,Y,26.121995,-80.122283,https://www.redfin.com/FL/Fort-Lauderdale/45-H...,Natacha Bednarczyk,Redfin Corporation,None,None,Natacha Bednarczyk,Redfin Corporation,None,None,1,orange,45 HENDRICKS CONDO at 45 Hendricks Isle #201 i...
1,PAST SALE,2024-10-24,Condo/Co-op,48 Hendricks Isle Unit 3A-A,Fort Lauderdale,FL,33301.0,1400000.0,3.0,3.5,The Meritage,2458.0,NaN,2006.0,NaN,570.0,1913.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Beaches MLS,F10433988,N,Y,26.122099,-80.121742,https://www.redfin.com/FL/Fort-Lauderdale/48-H...,Matthew Bertanzetti,Florida Luxurious Properties,None,None,Spencer Zorn,"Compass Florida, LLC.",None,None,2,blue,The Meritage at 48 Hendricks Isle Unit 3A-A in...
2,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #3808,Hallandale Beach,FL,33009.0,1085000.0,3.0,2.5,BEACH CLUB CONDO,1571.0,NaN,2005.0,NaN,691.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11650323,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Damira Liberman,LoKation,None,None,Boris Gelfand,"Gelfand Realty, Inc.",None,None,3,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3808 in H...
3,PAST SALE,2024-10-23,Condo/Co-op,1631 Riverview Rd #808,Deerfield Beach,FL,33441.0,800000.0,3.0,2.0,Hillsboro Landings Condo NO One,1824.0,NaN,1973.0,NaN,439.0,983.0,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1631...,Beaches MLS,RX-11006751,N,Y,26.313508,-80.084597,https://www.redfin.com/FL/Deerfield-Beach/1631...,Jill Kendall,RE/MAX Select Group,Judith Wheeler,RE/MAX Select Group,Gustavo Pavone,The Cove Guspav Realty,None,None,4,blue,Hillsboro Landings Condo NO One at 1631 Riverv...
4,PAST SALE,2024-10-24,Condo/Co-op,600 W Las Olas Blvd #703,Fort Lauderdale,FL,33312.0,735000.0,2.0,2.0,Symphony South,1348.0,NaN,2005.0,NaN,545.0,1397.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Beaches MLS,F10448050,N,Y,26.118290,-80.149165,https://www.redfin.com/FL/Fort-Lauderdale/600-...,Austin Bergman,"Compass Florida, LLC",None,None,None,None,None,None,5,blue,Symphony South at 600 W Las Olas Blvd #703 in ...
5,PAST SALE,2024-10-24,Condo/Co-op,1850 S Ocean Dr #2106,Hallandale Beach,FL,33009.0,700000.0,2.0,1.0,BEACH CLUB CONDO,1086.0,NaN,2005.0,NaN,645.0,1119.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11589107,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Juliana Tobon,Cervera Real Estate Inc,None,None,Marta Kottar,One Sotheby's International Realty,None,None,6,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #2106 in H...
6,PAST SALE,2024-10-21,Condo/Co-op,1201 S Ocean Dr Unit 2504N,Hollywood,FL,33019.0,692000.0,2.0,2.0,The Summit,1570.0,NaN,1982.0,NaN,441.0,1564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Beaches MLS,F10450988,N,Y,26.006228,-80.117346,https://www.redfin.com/FL/Hollywood/1201-S-Oce...,Jill Fugger,BW Oceanside,None,None,Mariene Menin,Coldwell Banker Realty,None,None,7,blue,The Summit at 1201 S Ocean Dr Unit 2504N in Ho...
7,PAST SALE,2024-10-24,Condo/Co-op,5200 N Ocean Blvd #1203,Lauderdale By The Sea,FL,33308.0,685000.0,2.0,2.0,Sea Ranch Lakes North,1330.0,NaN,1971.0,NaN,515.0,943.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/5200...,Beaches MLS,F10463114,N,Y,26.201000,-80.094629,https:

## Time on Market Calculator

In [68]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 12) ## List (Earlier) date
date2 = datetime(2024, 10, 24) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

165


## Clean CSV for Datawrapper Chart

In [69]:
chart_df = df_top_ten

In [70]:
chart_df = chart_df.fillna(" ")

In [71]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [72]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [73]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [74]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [78]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-10-22,45 Hendricks Isle #201 in Fort Lauderdale,"$1,800,000",3,3.5,"2,979",2007,$604,MARMLS,A11611277,Natacha Bednarczyk Redfin Corporation,Natacha Bednarczyk Redfin Corporation
1,2024-10-24,48 Hendricks Isle Unit 3A-A in Fort Lauderdale,"$1,400,000",3,3.5,"2,458",2006,$570,Beaches MLS,F10433988,Matthew Bertanzetti Florida Luxurious Properti...,"Spencer Zorn Compass Florida, LLC."
2,2024-10-24,1850 S Ocean Dr #3808 in Hallandale Beach,"$1,085,000",3,2.5,"1,571",2005,$691,MARMLS,A11650323,Damira Liberman LoKation,"Boris Gelfand Gelfand Realty, Inc."
3,2024-10-23,1631 Riverview Rd #808 in Deerfield Beach,"$800,000",3,2.0,"1,824",1973,$439,Beaches MLS,RX-11006751,Jill Kendall RE/MAX Select Group Judith Wheele...,Gustavo Pavone The Cove Guspav Realty
4,2024-10-24,600 W Las Olas Blvd #703 in Fort Lauderdale,"$735,000",2,2.0,"1,348",2005,$545,Beaches MLS,F10448050,"Austin Bergman Compass Florida, LLC",
5,2024-10-24,1850 S Ocean Dr #2106 in Hallandale Beach,"$700,000",2,1.0,"1,086",2005,$645,MARMLS,A11589107,Juliana Tobon Cervera Real Estate Inc,Marta Kottar One Sotheby's International Realt...
6,2024-10-21,1201 S Ocean Dr Unit 2504N in Hollywood,"$692,000",2,2.0,"1,570",1982,$441,Beaches MLS,F10450988,Jill Fugger BW Oceanside,Mariene Menin Coldwell Banker Realty
7,2024-10-24,5200 N Ocean Blvd #1203 in Lauderdale By The Sea,"$685,000",2,2.0,"1,330",1971,$515,Beaches MLS,F10463114,"Eldrin DiBello By The Sea Realty, Inc.","Eldrin DiBello By The Sea Realty, Inc."
8,2024-10-24,4106 W Palm Aire Dr Unit 63a in Pompano Beach,"$685,000",2,2.5,"2,586",1986,$265,Beaches MLS,RX-11010573,Andres Sandoval KW Innovations Amanda Tavares ...,Beth Daly RE/MAX Experience
9,2024-10-21,2700 NE 56th Ct Unit 2B in Fort Lauderdale,"$592,500",2,2.0,"1,400",1967,$423,MARMLS,A11659850,Evan Feig Keller Williams Elite Properties Nic...,Isabella Guzik London Foster Realty
